# Analyse, classification et indexation des données: feuille 9
### Apprentissage supervisé, Arbres de décision et Forêts aléatoires

## Avant propos

Dans cette feuille, nous allons étudier une catégorie d'algorithmes d'apprentissage supervisé différente de ce que nous avons vu jusque là : 

- les arbres de décisions ont cela de particulier qu'ils sont construits en utilisant des éléments de la théorie de l'information. Leur utilisation, ensuite, est assez facilement implémentable par des règles Si-Sinon simples. 

- les forêts aléatoires implémentent, elles, une approche de bagging.

In [4]:
import numpy as np
import pandas as pd
import warnings
import math
warnings.filterwarnings("ignore")

### Exercice 1. Arbres de décision (à la main)

Dans ce premier exercice, nous allons écrire un ensemble de fonctions implémentant les éléments vus en cours et permettant de construire un arbre de décision.  

Pour vérifier vos fonctions, vous pouvez les appliquer à l'exemple très simple vu en cours :


In [9]:
exemple_cours = pd.DataFrame(data={'A': [0, 0, 1, 1],
                                  'B': [1, 0, 1, 0],
                                  'Classe': ['C1', 'C1', 'C2', 'C2']})
exemple_cours

,A,B,Classe
0,0,1,C1
1,0,0,C1
2,1,1,C2
3,1,0,C2


1- Ecrire une fonction <code>information(dataset, label_feature)</code> permettant de calculer la quantité d'information nécessaire pour classifier un élément du corpus <code>dataset</code>. Le paramètre <code>label_feature</code> indique la colonne contenant les classes des éléments.

In [53]:
def information(dataset, label_feature):
    p = sum(dataset[label_feature] == "C1")
    n = sum(dataset[label_feature] == "C2")
    #print(f"nb de p = {p} | nb de n = {n}")
    if p == 0 or n == 0:
        return 0  # Entropie est 0 lorsque l'une des probabilités est 0

    total = p + n
    info = -p * math.log2(p / total) - n * math.log2(n / total)
    #print(f"information : {info}")
    return info

info_result = information(exemple_cours, "Classe")
print(f"info de dataset : {info_result}")


info de dataset : 4.0


2- Ecrire une fonction <code>entropy(dataset, feature, label_feature)</code> calculant l'entropie du descripteur <code>feature</code>

In [46]:
def entropy(dataset, feature, label_feature):
    entropy_result = 0
    # Compte le nombre de valeurs que peut prendre A 
    nvu = dataset[feature].nunique()
    for i in range(nvu):
        #Appel le dataset avec les ligne où A = i
        subset = dataset[dataset[feature] == i]
        entropy_result = entropy_result + int(information(subset, label_feature))
        # print(f"Entropie de {i} : {information(subset, label_feature)}")
        # print(f"Entropie de {i} : {entropy_result}")
    return entropy_result

etp = entropy(exemple_cours, "B", "Classe")
etp1 = entropy(exemple_cours, "A", "Classe")
print(f"Entropie de A : {etp1}")
print(f"Entropie de B : {etp}")

Entropie de A : 0
Entropie de B : 4


3- Ecrire une fonction <code>gain(dataset, feature, label_feature)</code> calculant le gain d'information obtenu en choisissant le descripteur <code>feature</code> pour partitionner le corpus <code>dataset</code>

In [62]:
def gain(dataset, feature, label_feature):
    return information(dataset, label_feature) - entropy(dataset, feature, label_feature)

print(f"Gain de A : {gain(exemple_cours, 'A', 'Classe')}")
print(f"Gain de B : {gain(exemple_cours, 'B', 'Classe')}")

Gain de A : 4.0
Gain de B : 0.0


4- En vous aidant des fonctions que vous avez écrites ci-dessus, construisez l'arbre de décision pour prédire Achat avec le corpus suivant. On ne demande pas d'écrire un programme qui construit l'arbre mais plutôt d'utiliser les fonctions pour faire les calculs à votre place.

\begin{array}{|c|c|c|c|c|}
\hline
\hline
Sexe & Age & Etat civil & Revenu & Achat \\
\hline
Homme & 18-35   & Marie       & Moyen  & Non \\
Homme & <18     & Celibataire & Faible & Non \\
Homme & >35     & Marie       & Eleve  & Oui \\
Femme & <18     & Celibataire & Moyen  & Non \\
Homme & 18 - 35 & Celibataire & Moyen  & Non \\
Femme & 18 - 35 & Celibataire & Eleve  & Oui \\
Femme & 18 - 35 & Marie       & Faible & Non \\
Homme & 18 - 35 & Marie       & Eleve  & Oui \\
Homme & >35     & Celibataire & Faible & Oui \\
Femme & <18     & Celibataire & Moyen  & Non \\
Femme & >35     & Celibataire & Moyen  & Oui \\
Femme & >35     & Marie       & Eleve  & Oui \\
Homme & 18 - 35 & Celibataire & Faible & Non\\
Femme & 18 - 35 & Marie       & Moyen  & Oui\\
\hline
\hline
\end{array}

In [64]:
dataset = pd.DataFrame(data={'Sexe': ['Homme', 'Homme', 'Homme', 'Femme','Homme', 'Femme', 'Femme', 
                                'Homme', 'Homme', 'Femme', 'Femme', 'Femme','Homme', 'Femme'],
                      'Age': ['18-35', '<18', '>35', '<18', '18-35', '18-35', '18-35', '18-35', 
                              '>35', '<18', '>35', '>35', '18-35', '18-35'],
                      'Etat_civil': ['marie', 'celibataire', 'marie', 'celibataire', 'celibataire', 
                                     'celibataire', 'marie', 'marie', 'celibataire', 'celibataire',
                                    'celibataire', 'marie', 'celibataire', 'marie'],
                      'Revenu': ['Moyen', 'Faible', 'Eleve', 'Moyen', 'Moyen', 'Eleve', 'Faible',
                                'Eleve', 'Faible', 'Moyen', 'Moyen', 'Eleve', 'Faible', 'Moyen'],
                      'Achat': ['Non', 'Non', 'Oui', 'Non', 'Non', 'Oui', 'Non', 'Oui', 'Oui', 'Non',
                               'Oui', 'Oui', 'Non', 'Oui']})
#dataset

In [65]:
print(information(dataset, "Achat"))

0


In [66]:
print(f"gain pour le SEX : {gain(dataset, 'Sexe', 'Achat')}")
print(f"gain pour le AGE : {gain(dataset, 'Age', 'Achat')}")
print(f"gain pour le ETAT CIVIL : {gain(dataset, 'Etat_civil', 'Achat')}")
print(f"gain pour le REVENU : {gain(dataset, 'Revenu', 'Achat')}")


gain pour le SEX : 0
gain pour le AGE : 0
gain pour le ETAT CIVIL : 0
gain pour le REVENU : 0


5- Dessiner l'arbre de décision obtenu.

## Exercice 2. Arbres de décision avec <code>sklearn</code>

Dans cet exercice, nous allons utiliser la bibliothèque <code>sklearn</code> pour entraîner et tester un arbre de décision. 

Pour cela, nous allons utiliser le corpus <code>titanic.csv</code>. Un échantillon de ce corpus est diponible à l'adresse : 

https://www.labri.fr/perso/zemmari/datasets/titanic.csv

Une description détaillée de ce corpus et des descripteurs le composant peut être consultée à l'adresse :

https://www.kaggle.com/c/titanic/data

On vous laisse découvrir la tragédie à laquelle est lié ce corpus :-)


1- Chargez le corpus et explorez son contenu.

2- Supprimez les lignes contenant des données manquantes

3- On souhaite construire un modèle pour prédire, pour un passager, s'il survivra ou non. Supprimez les descripteurs qui ne vous paraissent pas pertinents pour la prédiction. Expliquez vos choix.

4- La cellule suivante permet de convertir une catégorie au format entier ou au format texte en un entier. 
Exécutez la telle quelle et observez son résultat.

In [ ]:
from sklearn import preprocessing

cols = ['Sex', 'Embarked']
le = preprocessing.LabelEncoder()
for c in cols:
    le.fit(dataset[c])
    dataset[c] = le.transform(dataset[c])
dataset

5- Séparez les données en un ensemble $X$ contenant les variables explicatives et $y$ la variable à prédire.

6- Partitionnez les données en ensemble d'entraînement et de test

7- Instanciez un classifieur Decision Tree et entraînez-le avec les données d'entraînement.

8- Visualisez l'arbre obtenu. Décommentez/complétez pour cela le code suivant

In [ ]:
#from sklearn.tree import export_graphviz
#from six import StringIO  
#from IPython.display import Image  
#import pydotplus

#features_cols = ...
#class_names=['0','1']

#dot_data = StringIO()
#export_graphviz(dt, out_file=dot_data,  
#                filled=True, rounded=True,
#                special_characters=True,feature_names = features_cols,class_names=class_names)
#graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
#graph.write_png('titanic.png')
#Image(graph.create_png())

9- Mesurez la qualité de votre classifieur.

10- Changez les valeurs par défaut des différents paramètres et observez la qualité de votre classifieur. 

## Exercice 3. Random forest

Quelles sont les performances d'un classifieur de type forêts aléatoires sur le même corpus que l'exercice précédant ?



Cherchez le meilleur paramètrage pour le meilleur classifieur en complétant / décommentant le code suivant.

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
# n_estimators = ...
# Number of features to consider at every split
# max_features = ...
# Maximum number of levels in tree
# max_depth = ...
# Minimum number of samples required to split a node
# min_samples_split = ...
# Minimum number of samples required at each leaf node
# min_samples_leaf = ...
# Method of selecting samples for training each tree
# bootstrap = ...
# Create the random grid
#random_grid = {'n_estimators': n_estimators,
#               'max_features': max_features,
#               'max_depth': max_depth,
#               'min_samples_split': min_samples_split,
#               'min_samples_leaf': min_samples_leaf,
#              'bootstrap': bootstrap}
#print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, 
                               cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)

In [ ]:
rf_random.best_params_